In [9]:
'''
# 四章 NNの学習
- NNの学習 = 訓練データから最適な重みパラメータの値を自動で獲得すること
- NNが学習を行えるようにするため、損失関数を導入する
    - この値が最も小さくなる重みパラメータを探し出すのが学習の目的
    - まずは勾配法を導入する
- NNはデータから学習できることが特徴
- 機械学習はデータが中心
    - ex: パーセプトロン
    - データをまず用意する必要がある
    - また、推測を介さず、データからのみストーリーを語る    
- たとえば、ゼロから「5」を認識するアルゴリズムをひねり出すより、画像から特徴量を抽出して、そのパターンを学習する方法がある
    - 特徴量とは、入力データから、本質的なデータを的確に抽出できるように設計された「変換器」を指す
    - 画像の特徴量は一般的にベクトルで記述される
        - コンピュータビジョンで有名な特徴量としては、SIFT, SURF, HOGなどがある
    - 画像データを変換器　= 特徴量によってベクトルに変換し、そのベクトルに対して「識別器」で学習させる
        - [ ] 識別器とは
            - SVN, KNN等があるらしい
- この「特徴量」については人間が設計する必要がある
    - 問題に応じて特徴量 = 変換器を使い分ける必要がある
    - ココには人の手が介在する
- ここまでの推測の種類の生理
    1. 入力 -> 人間の考えたアルゴリズム -> 出力
    2. 入力 -> 人間の考えた特徴量 -> 機械学習 -> 出力
    3. 入力 -> NN(deep learning) -> 出力
- よって、NN(DL)はend-to-end-machine-learningと呼ばれることがある

## データの取扱について
- 機械学習の問題では、訓練データとテストデータの2つにデータを分けて学習やじっけんを行なう
    1. 訓練データで適切なパラメータを探索
    2. テストデータでモデルの実力を評価
- 汎化能力を正しく評価するためにデータを2つに分離する必要がある
    - 汎化能力 = まだ見ぬデータへの推測力
    - データセットが偏っていると、特定のデータセットにうまく対応できても、他のには対応できなくなると行った状況に陥る
    - これを過学習 = overfittingという

## 損失関数
- NN性能の悪さを示す指標
- 通常は二乗和誤差や、交差エントロピー誤差などが用いられる
- 二乗和誤差
    - E = 0.5 * ∑(yk - tk)^2 ... kは添字
    - y: NNの出力, tは教師データ
- 教師データにおいて、正解データを1, それ以外を0と置く表記をone-hot表現という
'''

import numpy as np

# chapter3の例
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # softmax関数の出力
t = [    0,       0,    1,     0,        0,   0,     0,    0,     0,    0] # 教師データ


def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)
    
# 「２」を正解とする
t = [    0,       0,    1,     0,        0,   0,     0,    0,     0,    0] 

# ex1: 2の確率が最も高いと弾き出すモデルの出力の例
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t)) # 0.097...

# ex2: 7の確率が最も高いと弾き出すモデルの出力の例
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t)) # 0.597...

# 確かにこの損失関数は誤っているex2の方でおおきな値を返している

'''
- 交差エントロピー誤差
    - E = - ∑(tk * log(yk)) ... kは添字
    - one-hot表現だとすると、tk=1、つまり正解ラベルのときだけ計算する
    - また、softmaxの出力値で正解の確率を仮に1と出していた時、損失は0になる = 優秀であると判断できる
'''

def cross_entropy_error(y, t):
    delta = 1e-7 # 微小な値
    return -np.sum(t * np.log(y + delta))

# np.log(0)を計算してしまうとnp.log(0)が-infとなり、エラーが起きてしまうため


# 「2」を正解とする
t = [    0,       0,    1,     0,        0,   0,     0,    0,     0,    0] 

# ex1: 2の確率が最も高いと弾き出すモデルの出力の例
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) # 0.510825457099

# ex2: 7の確率が最も高いと弾き出すモデルの出力の例
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) # 2.3025840929945458


'''
## ミニバッチ学習
- 全ての訓練データに対して損失関数を求め、その値をできるだけ小さくするようなパラメータを探し出す必要がある
- よって、先程までの一つの訓練データの損失関数を求める式の更に総和を求める
    - E = -(1/N) * ∑(∑(tnk * logynk)) ... n, kは添字
- しかし、MNISTのデータセットであっても60000万件あり、bigdataにもなればかなりのデータ量になる
    - 全てのデータを対象に損失関数を計算するのは非現実的
- そこで、訓練データからある枚数だけを選び出し、その塊ごとに学習を行なう
    - この選びだしたものをミニバッチ = 小さな塊という
    - [ ] 選び出す数 = batch_sizeの適切な決め方は？
'''

import sys, os
sys.append.path('./deep-learning-from-scratch')
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_labeｌ=True)

train_size = x_train.shape[0] # 60000
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
# ex: array([25286, 10861, 55404, 16791, 26595, 34692, 39286,  9747, 15529, 18955])
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

def cross_entropy_error(y, t):
    if y.ndim == 1:
        # reshape: n * m行列を、n' * m'行列に変形してくれる。ただし、要素数がおなじになるような変形に限る
        # つまりここでは1次元配列を1 * nの2次元行列に変更している [1,2] => [[1. 2]]
        t = t.reshape(1, t.size) 
        y = y.reshape(1, y.size)
        
        batch_size = y.shape[0] 
        return -np.sum(t * np.log(y)) / batch_size
    
    


0.510825457099
